In [1]:
from utils.data_set import get_data_json, extract_data_set_info
from utils.matrix import fix_diagonal_weight
from itertools import product
from model.model import BSS
from typing import Set, List, Tuple
import networkx as nx
import mip
import pandas as pd

In [2]:
try:
    data_set = get_data_json("../../data/ReggioEmilia/4ReggioEmilia30.json")

    (num_vertices, demands, vehicle_capacity, distance_matrix) = extract_data_set_info(
        data_set
    )

    fix_diagonal_weight(distance_matrix)

    V = set(range(num_vertices))
    A = {(i, j): distance_matrix[i, j] for i in V for j in V}
    m = 1
    Q = vehicle_capacity
    q = demands
    c = distance_matrix

    problem = BSS(V, A, m, Q, q, c)
except Exception as e:
    raise e

2024-08-28 08:47:38.464 | SUCCESS  | utils.data_set:get_data_json:23 - JSON file loaded correctly


In [3]:
class SubTourElimination(mip.ConstrsGenerator):
    def __init__(self, F: List[Tuple[int, int]], V: Set[int], x: List[List[mip.Var]]):
        self.F = F
        self.V = V
        self.x = x

    def generate_constrs(self, model: mip.Model, depth: int = 0, npass: int = 0):
        y, V, cp, G = model.translate(self.x), self.V, mip.CutPool(), nx.DiGraph()

        for u, v in [(i, j) for i, j in product(V, V) if i != j and y[i][j]]:
            G.add_edge(u, v, capacity=y[u][v].x)

        for u, v in self.F:
            cut_value, (S, NS) = nx.minimum_cut(G, u, v)
            if cut_value <= 0.99:
                a_ins = [
                    (y[i][j], y[i][j].x)
                    for (i, j) in product(V, V)
                    if i != j and y[i][j] and i in S and j in S and 0 not in S
                ]

                if sum(f for _, f in a_ins) >= len(S) - 1:
                    cut = mip.xsum(1.0 * v for v, _ in a_ins) <= len(S) - 1
                    cp.add(cut)
        for cut in cp.cuts:
            model += cut

In [4]:
problem.V, len(problem.A)

({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}, 196)

In [5]:
problem.Q, problem.q

(30, [0, -3, 1, 2, -5, 2, -10, -2, -2, 3, -1, -6, -9, 2])

In [6]:
df = pd.DataFrame(problem.c)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,3700,4000,3900,4900,3900,4800,3800,4000,2600,4600,3800,3000,3500
1,4300,0,1300,300,2400,1200,1300,2600,1600,1200,1000,600,1400,800
2,4500,900,0,900,2100,900,1000,1900,1300,2100,700,1400,2200,500
3,4500,300,1300,0,2200,1200,1000,2500,1600,1500,700,800,1600,800
4,5500,2200,2600,2000,0,3100,1200,3600,3200,2500,1800,2000,2600,2700
5,3700,800,900,1100,2900,0,1800,1500,500,2000,1500,1300,1700,400
6,5300,1400,1800,1100,2000,2300,0,2700,2300,2200,900,1800,2400,1900
7,3500,2200,1700,2400,3600,1500,2400,0,1400,3000,2400,2600,2700,1800
8,4300,800,1100,1000,3100,1000,2000,1500,0,1600,1700,1200,1300,600
9,3300,1600,2600,1800,2300,2500,2200,3000,2700,0,2100,1200,800,2100


In [7]:
problem.f = {
    (i, j): problem.model.add_var(name=f"f_{i}_{j}", var_type=mip.INTEGER)
    for i in V
    for j in V
}

In [8]:
problem.model += mip.xsum(problem.f[i, i] for i in V) == 0

for j in V - {0}:
    problem.model += (
        mip.xsum(problem.f[j, i] for i in V if i != j)
        - mip.xsum(problem.f[i, j] for i in V if i != j)
        == q[j]
    )

for i in V:
    for j in V:
        if i != j:
            flow_lower_bound = max(0, q[i], -q[j]) * problem.x[i][j]
            flow_upper_bound = min(Q, Q + q[i], Q - q[j]) * problem.x[i][j]

            problem.model += problem.f[i, j] >= flow_lower_bound
            problem.model += problem.f[i, j] <= flow_upper_bound

In [9]:
F, G = [], nx.DiGraph()

for i, j in A:
    G.add_edge(i, j, weight=int(c[i, j]))

for i in V:
    pred, dist = nx.dijkstra_predecessor_and_distance(G, source=i)
    ds = list(dist.items())
    ds.sort(key=lambda x: x[1])
    F.append((i, ds[-1][0]))

In [10]:
problem.model.cuts_generator = SubTourElimination(F, V, problem.x)
problem.model.optimize()


Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 53 small elements
Coin0506I Presolve 352 (-57) rows, 364 (-28) columns and 1298 (-147) elements
Clp1000I sum of infeasibilities 0.323258 - average 0.000918347, 103 fixed columns
Coin0506I Presolve 238 (-114) rows, 235 (-129) columns and 923 (-375) elements
Clp0029I End of values pass after 235 iterations
Clp0014I Perturbing problem by 0.001% of 4.8101535 - largest nonzero change 5.5708976e-05 ( 0.00096646154%) - largest zero change 2.982246e-05
Clp0000I Optimal - objective value 18187.381
Clp0000I Optimal - objective value 18187.381
Coin0511I After Postsolve, objective 18187.381, infeasibilities - dual 0 (0), primal 0 (0)
Clp0014I Perturbing problem by 0.001% of 5.1837674 - largest nonzero change 5.3569463e-05 ( 0.00082704334%) - largest zero change 2.982246e-05
Clp0000I Optimal - object

<OptimizationStatus.OPTIMAL: 0>

In [11]:
if problem.model.num_solutions:
    route = [(i, j) for i in V for j in V if problem.x[i][j].x >= 0.5]
    flow = [
        (i, j) for i in V for j in V if problem.f[i, j].x and problem.x[i][j].x >= 0.5
    ]

    for i, j in route:
        print(f"{problem.model.var_by_name(f"x_{i}_{j}")} = {problem.x[i][j].x}")

x_0_3 = 1.0
x_1_11 = 1.0
x_2_10 = 1.0
x_3_1 = 1.0
x_4_6 = 1.0
x_5_8 = 1.0
x_6_3 = 1.0
x_7_5 = 1.0
x_8_13 = 1.0
x_9_7 = 1.0
x_10_4 = 1.0
x_11_12 = 1.0
x_12_0 = 1.0
x_12_9 = 1.0
x_13_2 = 1.0


In [12]:
for i, j in flow:
    print(f"{problem.model.var_by_name(f"f_{i}_{j}")} = {problem.f[i,j].x}")

f_0_3 = 28.0
f_1_11 = 27.0
f_2_10 = 16.0
f_3_1 = 30.0
f_4_6 = 10.0
f_5_8 = 15.0
f_7_5 = 13.0
f_8_13 = 13.0
f_9_7 = 15.0
f_10_4 = 15.0
f_11_12 = 21.0
f_12_9 = 12.0
f_13_2 = 15.0


In [13]:
problem.model.status, problem.model.objective_value

(<OptimizationStatus.OPTIMAL: 0>, 20400.0)